In [1]:
using Gridap
domain = (-1, +1, -1, +1)
partition = (20, 20)
model = CartesianDiscreteModel(domain, partition)

CartesianDiscreteModel()

In [2]:
# include("Julia_functions/solution_animation.jl")
# include("Julia_functions/indicator_chi.jl")

In [3]:
function χ(x, a, b)
    if a < x && x < b
        return 1
    else
        return 0
    end
end

χ (generic function with 1 method)

In [6]:
order = 1
reffe = ReferenceFE(lagrangian, Float64, order)
V0 = TestFESpace(model, reffe, conformity=:H1)

Ug = TransientTrialFESpace(V0)

degree = 2
Ω = Triangulation(model)
dΩ = Measure(Ω, degree)

Γ = BoundaryTriangulation(model)                        # triangulate the boundary ∂Ω
dΓ = Measure(Γ,degree)  


q_pos(x) = χ(x[1], -0.9, -0.7) * χ(x[2], 0, 0.20) + χ(x[1], 0.2, 0.4) * χ(x[2], -0.5, 0)

α(t) = x -> 1.0
f(t) = x -> 0.0
Tout(x,t)= 4.0
Tout(t)=x->Tout(x,t)

# ∫ ̇u*v-∇⋅(α(t)∇u)*v dΩ = ∫ Qv dΩ , -∫ α(t)∇u⋅n*v dΓ = ∫ (Tout(t)-u) v dΓ
m(t, dtu, v) = ∫(v * dtu)dΩ
a(t, u, v) = ∫(α(t) * ∇(v) ⋅ ∇(u))dΩ + ∫(u*v)dΓ
l(t, v) = ∫(q_pos*v * f(t))dΩ + ∫(Tout(t) * v)dΓ

op = TransientLinearFEOperator((a, m), l, Ug, V0)

op_opt = TransientLinearFEOperator((a, m), l, Ug, V0, constant_forms=(true, true))

ls = LUSolver()
Δt = 0.01
θ = 0.5
solver = ThetaMethod(ls, Δt, θ)

t0, tF = 0.0, 10.0
uh0 = interpolate_everywhere(20.0, Ug(t0))
uh = solve(solver, op, t0, tF, uh0)

GenericTransientFESolution()

In [7]:
if !isdir("tmp")
    mkdir("tmp")
  end
  
  createpvd("results") do pvd
    pvd[0] = createvtk(Ω, "tmp/results_0" * ".vtu", cellfields=["u" => uh0])
    for (tn, uhn) in uh
      pvd[tn] = createvtk(Ω, "tmp/results_$tn" * ".vtu", cellfields=["u" => uhn])
    end
  end

1003-element Vector{String}:
 "results.pvd"
 "tmp/results_0.vtu"
 "tmp/results_0.01.vtu"
 "tmp/results_0.02.vtu"
 "tmp/results_0.03.vtu"
 "tmp/results_0.04.vtu"
 "tmp/results_0.05.vtu"
 "tmp/results_0.060000000000000005.vtu"
 "tmp/results_0.07.vtu"
 "tmp/results_0.08.vtu"
 ⋮
 "tmp/results_9.929999999999833.vtu"
 "tmp/results_9.939999999999833.vtu"
 "tmp/results_9.949999999999832.vtu"
 "tmp/results_9.959999999999832.vtu"
 "tmp/results_9.969999999999832.vtu"
 "tmp/results_9.979999999999832.vtu"
 "tmp/results_9.989999999999831.vtu"
 "tmp/results_9.999999999999831.vtu"
 "tmp/results_10.009999999999831.vtu"